# Threat Hunting with Jupyter Notebook: Annoying FTP User Scenario

We're taking a look at FTP Bro log. Bro log is a stream of high level entries that correspond to lower level events. A bro log examples are including HTTP request/reply pair, email sent using SMTP, a login over SSH, an FTP command, an so on.

I am using an FTP Zeek log example generated from MACCDC2012 from [SecRepo](http://www.secrepo.com/) curated by Mike Sconzo with CC-BY license.

Let's build a case (hypothetically) and hypothesis around this log!

## Scenario
_Recently, you were informed that the Company internal FTP Server was down for quite some time. It's not the first time, nor the second time, but the third time it's happening this week. The FTP Server allowed anonymous login, so apparently it's not that obvious who made the server down. Fortunately you have your Zeek log and Python ready to investigate who might have downed the server._

### Hypothesis 1
The incident happened internally, so an employee might have DDoS'd the server unintentionally. It can be proven by looking at which IP address is using FTP abnormally, in other words, accessing FTP so often in short period of time.

First thing to do is to convert `ftp.log` into a readable csv file. I found a useful script to do that from [here](https://github.com/cyberdefendersprogram/MachineLearning)

In [37]:
import csv
import os
dic = {"ftp.log":["ts", "uid", "id_orig_h", "id_orig_p", "id_resp_h", "id_resp_p", "user", "password", "command", "arg","mime_type", "file_size", "reply_code", "reply_msg", "passive", "orig_h", "resp_h", "resp_p", "fuid"]}

path = "."
for filename in os.listdir(path):
    if filename not in dic:
        continue
    with open(path+"/"+filename.replace("log","csv"), 'w+', encoding='utf-8', newline='') as csvfile:
        w = csv.writer(csvfile, dialect='excel')
        with open(path+"/"+filename, encoding="utf8") as file:
            lines = file.read().split('\n')
            lines=lines[:-1]
            # print(lines)
            files = [dic[filename]]
        for line in lines:
            cells = []
            for item in line.split('\t'):
                if item=="-":
                    cells.append(item.replace("-",""))
                else:
                    cells.append(item)
            files.append(cells)
        w.writerows(files)

There is no print output from the script, but we now got the CSV file of the FTP to ease the `pandas` digestion.

In [38]:
import pandas as pd

df = pd.read_csv("ftp.csv")

#DataFrame with columns
columns = pd.DataFrame(list(df.columns.values[1:]))

#DataFrame with data types
data_types = pd.DataFrame(df.dtypes, columns=['Data Type'])

#DataFrame with Count
data_count = pd.DataFrame(df.count(), columns=['Count'])

#DataFrame with unique values
unique_value_counts = pd.DataFrame(columns=['Unique Values'])
for v in list(df.columns.values):
    unique_value_counts.loc[v] = [df[v].nunique()]

missing_data_counts = pd.DataFrame(df.isnull().sum(), columns=['Missing Values'])
ftp_digestion_report = data_types.join(data_count).join(unique_value_counts).join(missing_data_counts)
print('FTP Digestion Report')
ftp_digestion_report

FTP Digestion Report


,Data Type,Count,Unique Values,Missing Values
ts,float64,5796,2390,0
uid,object,5796,137,0
id_orig_h,object,5796,15,0
id_orig_p,int64,5796,95,0
id_resp_h,object,5796,21,0
id_resp_p,int64,5796,1,0
user,object,5796,4,0
password,object,5745,12,51
command,object,5796,6,0
arg,object,2966,1545,2830


There are ~5000 entries to be processed. We can start looking around.

In [39]:
df.head()

,ts,uid,id_orig_h,id_orig_p,id_resp_h,id_resp_p,user,password,command,arg,mime_type,file_size,reply_code,reply_msg,passive,orig_h,resp_h,resp_p,fuid
0,1.331904e+09,CNFo204HUpVHDn1qt2,192.168.203.45,34433,192.168.21.101,21,anonymous,IEUser@,PASV,NaN,NaN,NaN,227.0,"Entering Passive Mode (192,168,21,101,219,204).",T,192.168.203.45,192.168.21.101,56268.0,NaN
1,1.331904e+09,CyHkLo2YfhjddpbSVl,192.168.203.45,56158,192.168.21.103,21,anonymous,IEUser@,PASV,NaN,NaN,NaN,227.0,"Entering Passive Mode (192,168,21,103,192,28)",T,192.168.203.45,192.168.21.103,49180.0,NaN
2,1.331904e+09,CotBpLi55vt2fNqm7,192.168.202.96,40138,192.168.28.101,21,<unknown>,NaN,PORT,\x1d\x93!\xf8t\x1ck\xd64\x05\xbb\xbeyu$\x152\x...,NaN,NaN,220.0,ProFTPD 1.3.4rc2 Server (Debian) [::ffff:172.1...,NaN,NaN,NaN,NaN,NaN
3,1.331904e+09,CptK3340W66OKHK3Rd,192.168.202.96,43740,192.168.28.103,21,<unknown>,NaN,PORT,\xbe\xb9wN\x11\xd1\xe1yH\xb8fI\x86\xfdvG\x80\x...,NaN,NaN,530.0,Please log in with USER and PASS first.,NaN,NaN,NaN,NaN,NaN
4,1.331904e+09,C3NlQu4G9w4W3TGSj7,192.168.204.45,50584,192.168.21.101,21,anonymous,IEUser@,PASV,NaN,NaN,NaN,227.0,"Entering Passive Mode (192,168,21,101,163,245).",T,192.168.204.45,192.168.21.101,41973.0,NaN


So we got a lot of informations. We will start looking at the unique IP addresses accessing this server.

In [40]:
df.id_orig_h.unique()

array(['192.168.203.45', '192.168.202.96', '192.168.204.45',
       '192.168.24.100', '192.168.27.100', '192.168.202.102',
       '192.168.202.108', '192.168.202.118', '192.168.202.100',
       '192.168.202.79', '192.168.25.254', '192.168.202.94',
       '192.168.25.100', '192.168.202.138', '192.168.202.140'],
      dtype=object)

We got a list of IPs. Then we can count how many times each individual IP accessed the server.

In [42]:
for ip in df.id_orig_h.unique():
    ipobj = df.apply(lambda x: True if x['id_orig_h'] == ip else False, axis=1)
    numofaccess = len(ipobj[ipobj == True].index)
    print("IP", str(ip), "status:", str(numofaccess), "("+"{:.4f}".format(numofaccess/df.shape[0]*100), "%)")

IP 192.168.203.45 status: 4 (0.0690 %)
IP 192.168.202.96 status: 4 (0.0690 %)
IP 192.168.204.45 status: 14 (0.2415 %)
IP 192.168.24.100 status: 2 (0.0345 %)
IP 192.168.27.100 status: 4 (0.0690 %)
IP 192.168.202.102 status: 5478 (94.5135 %)
IP 192.168.202.108 status: 7 (0.1208 %)
IP 192.168.202.118 status: 5 (0.0863 %)
IP 192.168.202.100 status: 1 (0.0173 %)
IP 192.168.202.79 status: 4 (0.0690 %)
IP 192.168.25.254 status: 2 (0.0345 %)
IP 192.168.202.94 status: 209 (3.6059 %)
IP 192.168.25.100 status: 4 (0.0690 %)
IP 192.168.202.138 status: 55 (0.9489 %)
IP 192.168.202.140 status: 3 (0.0518 %)


Oh boy, now we're talking. We can see that the `192.168.202.102` IP was accessing the server a bunch of times, that the server became slow and basically inaccessible for the others. Let's take a look at what he/she's doing.

In [35]:
df.query("id_orig_h == '192.168.202.102'")

,ts,uid,id_orig_h,id_orig_p,id_resp_h,id_resp_p,user,password,command,arg,mime_type,file_size,reply_code,reply_msg,passive,orig_h,resp_h,resp_p,fuid
12,1.331905e+09,Cup8D83JUM166udWb,192.168.202.102,4379,192.168.21.101,21,ftp,password@example.com,DELE,ftp://192.168.21.101/.ftpduBnga4,NaN,NaN,550.0,/.ftpduBnga4: Operation not permitted,NaN,NaN,NaN,NaN,NaN
13,1.331905e+09,Cup8D83JUM166udWb,192.168.202.102,4379,192.168.21.101,21,ftp,password@example.com,PASV,NaN,NaN,NaN,227.0,"Entering Passive Mode (192,168,21,101,229,12).",T,192.168.202.102,192.168.21.101,58636.0,NaN
14,1.331905e+09,Cup8D83JUM166udWb,192.168.202.102,4379,192.168.21.101,21,ftp,password@example.com,STOR,ftp://192.168.21.101/.ftpduBnga4,NaN,NaN,550.0,/.ftpduBnga4: Operation not permitted,NaN,NaN,NaN,NaN,NaN
15,1.331905e+09,Cup8D83JUM166udWb,192.168.202.102,4379,192.168.21.101,21,ftp,password@example.com,PASV,NaN,NaN,NaN,227.0,"Entering Passive Mode (192,168,21,101,185,213).",T,192.168.202.102,192.168.21.101,47573.0,NaN
16,1.331905e+09,Cup8D83JUM166udWb,192.168.202.102,4379,192.168.21.101,21,ftp,password@example.com,DELE,ftp://192.168.21.101/.cache/.ftpduBnga4,NaN,NaN,550.0,/.cache/.ftpduBnga4: Operation not permitted,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5791,1.332016e+09,CqEHJM1FdQDlKSKHdb,192.168.202.102,1146,192.168.23.103,21,<unknown>,NaN,APPE,ftp://192.168.23.103/./\x83\xc7<\xbe\xf0]\xbd\...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5792,1.332016e+09,Ce5EAJ3fvdomukhjqc,192.168.202.102,1193,192.168.23.103,21,<unknown>,NaN,APPE,ftp://192.168.23.103/./\x83\xc7<\xbe\xf0]\xbd\...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5793,1.332016e+09,CYIJjk1jzXF2SQDqH1,192.168.202.102,1193,192.168.23.103,21,<unknown>,NaN,APPE,ftp://192.168.23.103/./\x83\xc7<\xbe\xf0]\xbd\...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5794,1.332016e+09,CzwwlL2TLNeRsKCGEh,192.168.202.102,1169,192.168.23.103,21,<unknown>,NaN,APPE,ftp://192.168.23.103/./\x83\xc7<\xbe\xf0]\xbd\...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
df.query("id_orig_h == '192.168.202.102'")[10:30]

,ts,uid,id_orig_h,id_orig_p,id_resp_h,id_resp_p,user,password,command,arg,mime_type,file_size,reply_code,reply_msg,passive,orig_h,resp_h,resp_p,fuid
22,1.331905e+09,Cup8D83JUM166udWb,192.168.202.102,4379,192.168.21.101,21,ftp,password@example.com,STOR,ftp://192.168.21.101/.ssh/.ftpduBnga4,NaN,NaN,550.0,/.ssh/.ftpduBnga4: Operation not permitted,NaN,NaN,NaN,NaN,NaN
23,1.331905e+09,Cup8D83JUM166udWb,192.168.202.102,4379,192.168.21.101,21,ftp,password@example.com,PASV,NaN,NaN,NaN,227.0,"Entering Passive Mode (192,168,21,101,227,214).",T,192.168.202.102,192.168.21.101,58326.0,NaN
24,1.331905e+09,Cup8D83JUM166udWb,192.168.202.102,4379,192.168.21.101,21,ftp,password@example.com,DELE,ftp://192.168.21.101/dept/.ftpduBnga4,NaN,NaN,550.0,/dept/.ftpduBnga4: Operation not permitted,NaN,NaN,NaN,NaN,NaN
25,1.331905e+09,Cup8D83JUM166udWb,192.168.202.102,4379,192.168.21.101,21,ftp,password@example.com,PASV,NaN,NaN,NaN,227.0,"Entering Passive Mode (192,168,21,101,160,231).",T,192.168.202.102,192.168.21.101,41191.0,NaN
26,1.331905e+09,Cup8D83JUM166udWb,192.168.202.102,4379,192.168.21.101,21,ftp,password@example.com,STOR,ftp://192.168.21.101/dept/.ftpduBnga4,NaN,NaN,550.0,/dept/.ftpduBnga4: Operation not permitted,NaN,NaN,NaN,NaN,NaN
27,1.331905e+09,Cup8D83JUM166udWb,192.168.202.102,4379,192.168.21.101,21,ftp,password@example.com,PASV,NaN,NaN,NaN,227.0,"Entering Passive Mode (192,168,21,101,211,12).",T,192.168.202.102,192.168.21.101,54028.0,NaN
28,1.331905e+09,Cup8D83JUM166udWb,192.168.202.102,4379,192.168.21.101,21,ftp,password@example.com,DELE,ftp://192.168.21.101/dept/env/.ftpduBnga4,NaN,NaN,550.0,/dept/env/.ftpduBnga4: Operation not permitted,NaN,NaN,NaN,NaN,NaN
29,1.331905e+09,Cup8D83JUM166udWb,192.168.202.102,4379,192.168.21.101,21,ftp,password@example.com,PASV,NaN,NaN,NaN,227.0,"Entering Passive Mode (192,168,21,101,204,199).",T,192.168.202.102,192.168.21.101,52423.0,NaN
30,1.331905e+09,Cup8D83JUM166udWb,192.168.202.102,4379,192.168.21.101,21,ftp,password@example.com,STOR,ftp://192.168.21.101/dept/env/.ftpduBnga4,NaN,NaN,550.0,/dept/env/.ftpduBnga4: Operation not permitted,NaN,NaN,NaN,NaN,NaN
31,1.331905e+09,Cup8D83JUM166udWb,192.168.202.102,4379,192.168.21.101,21,ftp,password@example.com,PASV,NaN,NaN,NaN,227.0,"Entering Passive Mode (192,168,21,101,192,71).",T,192.168.202.102,192.168.21.101,49223.0,NaN


You can see on the `reply_msg` that he/she failed to store files a bunch of times. Perhaps we should ask him/her if he's running a downloaded script to upload data without editing the script. He/she might have done this unintentionally.

## Conclusion
What we have learned in this notebook:
- Converting FTP Zeek log file to a pandas-digestable CSV file
- Reading unique value of each column in the log file
- Counting how many times each unique value comes out

The next step should be validating the IP and correlating IP and the employee using it.

## Epilogue
While this is only a hypothetical case, this blog summarize how to find unique user and how many times the occurence in Jupyter. There are bunch of other things we can do with pandas such as filtering, counting, and looking for statistically-outliers.

Jupyter Notebook is a useful tool to demonstrate our logic and how we do it without us being with the reader.